# Team information

|Team-number :| 1|
|:----:|:----:|


|Name|    E-Mail        |matriculation-nr.|
|:----:|:----:|:----:|
|Tamara Scherer| schere21@ads.uni-passau.de|104218|
|Felix Müller| muell518@ads.uni-passau.de|104227|


# Import

In [ ]:
#from label_studio_sdk import Client
#from label_studio_sdk import project
#from label_studio_sdk import project
#import pandas as pd
#LABEL_STUDIO_URL = 'http://132.231.59.226:8080' #this address needs to be the same as the address the label-studio is hosted on.
#API_KEY = '1655a8922f821195356a17a3224c0532b091c61d' #please add your personal API_Key here to get your API_Key follow the Pictures below

#ls = Client(url=LABEL_STUDIO_URL, api_key=API_KEY)
#ls.check_connection()
#pro = project.Project.get_from_id(ls,"1")
#tasks = project.Project.get_labeled_tasks(pro)
#tasks[0]

In [1]:
# import labeled data
import pickle
with open("Files/tasks3", 'rb') as f:
    tasks = pickle.load(f)

In [2]:
# import necessary packages
import re
import pandas as pd

In [192]:
# filter the necessary texts with the corresponding label
df = pd.DataFrame(columns = ['Text', 'Label',])

for i in range(len(tasks)): 
    for j in range(len(tasks[i]['annotations'][0]['result'])): 
        df = df.append({
            'Text' : tasks[i]['annotations'][0]['result'][j]['value']['text'],
            'Label' : tasks[i]['annotations'][0]['result'][j]['value']['labels']
                        }, ignore_index = True)
df

,Text,Label
0,Good afternoon and thanks a lot for taking my ...,[QID_1]
1,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific]
2,Good afternoon and thanks a lot for taking my ...,[Question_2_specific]
3,Good afternoon and thanks a lot for taking my ...,[Question_3_neutral]
4,"No, I think that as it relates to the one, I t...",[AID_1]
...,...,...
11289,One of the areas obviously which has been a st...,[Question_3_neutral]
11290,"Well, let's start with the first part of your ...",[AID_1]
11291,"Well, let's start with the first part of your ...",[Answer_1_specific]
11292,"Well, let's start with the first part of your ...",[Answer_2_positive]


In [193]:
# assign a unique number to each label
def categorise(row):  
    if row['Label'] == ['Question_1_Company_specific']:
        return '1'
    elif row['Label'] == ['Question_1_Market_related']:
        return '2'
    elif row['Label'] == ['Question_2_specific']:
        return '3'
    elif row['Label'] == ['Question_2_open']:
        return '4'
    elif row['Label'] == ['Question_3_attack']:
        return '5'
    elif row['Label'] == ['Question_3_support']:
        return '6'
    elif row['Label'] == ['Question_3_neutral']:
        return '7'
    elif row['Label'] == ['Answer_1_specific']:
        return '8'
    elif row['Label'] == ['Answer_1_avoid_excuse']:
        return '9'
    elif row['Label'] == ['Answer_2_positive']:
        return '10'
    elif row['Label'] == ['Answer_2_negative']:
        return '11'
    elif row['Label'] == ['Answer_3_blame']:
        return '12'
    elif row['Label'] == ['Answer_3_no_blame']:
        return '13'
    else:
        return '0'
    
    
# call function and write results in a new column of the dataframe
df['LabelNumber'] = df.apply(lambda row: categorise(row), axis=1)

df

,Text,Label,LabelNumber
0,Good afternoon and thanks a lot for taking my ...,[QID_1],0
1,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1
2,Good afternoon and thanks a lot for taking my ...,[Question_2_specific],3
3,Good afternoon and thanks a lot for taking my ...,[Question_3_neutral],7
4,"No, I think that as it relates to the one, I t...",[AID_1],0
...,...,...,...
11289,One of the areas obviously which has been a st...,[Question_3_neutral],7
11290,"Well, let's start with the first part of your ...",[AID_1],0
11291,"Well, let's start with the first part of your ...",[Answer_1_specific],8
11292,"Well, let's start with the first part of your ...",[Answer_2_positive],10


**Filter one ID stage (e.g. Question_1_XX)**

In [194]:
idStage = ['1', '2']
df = df[df['LabelNumber'].isin(idStage)]

# set new index
df = df.reset_index()
# delete old indices
df.pop('index')
df

,Text,Label,LabelNumber
0,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1
1,"Okay, that's very helpful. And then on your gr...",[Question_1_Company_specific],1
2,"Hi Richard. So, on gross margin, it looked pre...",[Question_1_Market_related],2
3,And the core-on-core was pretty normal for you...,[Question_1_Company_specific],1
4,Okay. And my follow-up is on the EBIT dollar ...,[Question_1_Company_specific],1
...,...,...,...
1304,Hi. This is Mark for Pat. Thank you so much fo...,[Question_1_Company_specific],1
1305,Yes. I'm just wondering if you could talk a li...,[Question_1_Company_specific],1
1306,"Hi, thank you. Congrats on the quarter. Just a...",[Question_1_Company_specific],1
1307,Thanks for squeezing me in guys. This question...,[Question_1_Company_specific],1


# Preprocessing

In [ ]:
pip install gensim

In [186]:
# necessary imports for preprocessing steps

import string
import nltk

# import a stemmer for english words
snowStem = nltk.stem.SnowballStemmer('english')

# import stopwords
from nltk.corpus import stopwords
en_stopwords = stopwords.words('english')

# import for tokenization
from nltk import word_tokenize
nltk.download('punkt')

# import for lemmatizer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize,pos_tag
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ts23\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [195]:
# function that includes all preprocessing steps

def do_preprocessing(text_to_clean, remove_stopwords = True, stemming = True):
    # remove numbers 
    text_to_clean = re.sub(r'\d+', '', text_to_clean)
    # remove leading and ending white spaces
    text_to_clean = text_to_clean.strip()
    # transform text to lower case
    text_to_clean = text_to_clean.lower()

    if stemming:
      # stemming
      text_to_clean = snowStem.stem(text_to_clean)
    
    if remove_stopwords:
        # remove stop words
        text_to_clean = ' '.join([w for w in text_to_clean.split() if not(w in en_stopwords)])

    # remove punctuation
    text_to_clean = text_to_clean.translate(str.maketrans('','', string.punctuation))
    
    return text_to_clean


# call function and write results in new column of the dataframe
df.loc[:, 'CleanText'] = df['Text'].apply(
    lambda x: do_preprocessing(x, True, True))

df

,Text,Label,LabelNumber,CleanText
0,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1,good afternoon thanks lot taking question so r...
1,"Okay, that's very helpful. And then on your gr...",[Question_1_Company_specific],1,okay thats helpful growth china surpass expect...
2,"Hi Richard. So, on gross margin, it looked pre...",[Question_1_Market_related],2,hi richard so gross margin looked pretty solid...
3,And the core-on-core was pretty normal for you...,[Question_1_Company_specific],1,coreoncore pretty normal well
4,Okay. And my follow-up is on the EBIT dollar ...,[Question_1_Company_specific],1,okay followup ebit dollar growth looked like c...
...,...,...,...,...
1304,Hi. This is Mark for Pat. Thank you so much fo...,[Question_1_Company_specific],1,hi mark pat thank much taking question could t...
1305,Yes. I'm just wondering if you could talk a li...,[Question_1_Company_specific],1,yes im wondering could talk little bit custome...
1306,"Hi, thank you. Congrats on the quarter. Just a...",[Question_1_Company_specific],1,hi thank you congrats quarter followup toms qu...
1307,Thanks for squeezing me in guys. This question...,[Question_1_Company_specific],1,thanks squeezing guys question you john know g...


In [196]:
# function for lemmatization
def lemmatization(text):
    
  text = word_tokenize(text)
  result=[]
  wordnet = WordNetLemmatizer()
  for token,tag in pos_tag(text):
        pos=tag[0].lower()
        
        if pos not in ['a', 'r', 'n', 'v']:
            pos='n'
            
        result.append(wordnet.lemmatize(token,pos))

  return result

# call function
df['CleanText'] = df['CleanText'].apply(
    lambda x: lemmatization(x))

df

,Text,Label,LabelNumber,CleanText
0,Good afternoon and thanks a lot for taking my...,[Question_1_Company_specific],1,"[good, afternoon, thanks, lot, take, question,..."
1,"Okay, that's very helpful. And then on your gr...",[Question_1_Company_specific],1,"[okay, thats, helpful, growth, china, surpass,..."
2,"Hi Richard. So, on gross margin, it looked pre...",[Question_1_Market_related],2,"[hi, richard, so, gross, margin, look, pretty,..."
3,And the core-on-core was pretty normal for you...,[Question_1_Company_specific],1,"[coreoncore, pretty, normal, well]"
4,Okay. And my follow-up is on the EBIT dollar ...,[Question_1_Company_specific],1,"[okay, followup, ebit, dollar, growth, look, l..."
...,...,...,...,...
1304,Hi. This is Mark for Pat. Thank you so much fo...,[Question_1_Company_specific],1,"[hi, mark, pat, thank, much, take, question, c..."
1305,Yes. I'm just wondering if you could talk a li...,[Question_1_Company_specific],1,"[yes, im, wondering, could, talk, little, bit,..."
1306,"Hi, thank you. Congrats on the quarter. Just a...",[Question_1_Company_specific],1,"[hi, thank, you, congrats, quarter, followup, ..."
1307,Thanks for squeezing me in guys. This question...,[Question_1_Company_specific],1,"[thanks, squeeze, guy, question, you, john, kn..."


In [197]:
# convert tokens back to strings for further processing
string1=" "
df['CleanText'] = df['CleanText'].apply(
    lambda x: string1.join(x))

df

# Split of training and test dataset

In [204]:
import numpy as np

training_fraction = 0.70
training_size = int(np.floor(len(df) * training_fraction))

data_train, data_test = df[:training_size], df[training_size:]
#X_train, X_test = data_train.drop(['monthly_stock_return'], axis = 1), data_test.drop(['monthly_stock_return'], axis = 1)
#y_train, y_test = data_train[['monthly_stock_return']], data_test[['monthly_stock_return']]

# Bag-of-Words (BOW)

In [206]:
from sklearn.feature_extraction.text import CountVectorizer

def do_bow(data):
    count_vec = CountVectorizer()

    bow = count_vec.fit_transform(data['CleanText'])
    bow_matrix = pd.DataFrame(data = bow.toarray(), columns = count_vec.get_feature_names_out())

    #display(bow_matrix)
    
    return(bow_matrix)

# call function
train_bow_matrix = do_bow(data_train)
test_bow_matrix = do_bow(data_test)

# TF-IDF

In [207]:
from sklearn.feature_extraction.text import TfidfVectorizer

def do_tfidf(data):
    tfidf_vec = TfidfVectorizer()

    tfidf = tfidf_vec.fit_transform(data['CleanText'])
    tfidf_matrix = pd.DataFrame(data = tfidf.toarray(), columns = tfidf_vec.get_feature_names_out())

    #display(tfidf_matrix)
    
    return(tfidf_matrix)

# call function
train_tfidf_matrix = do_tfidf(data_train)
test_tfidf_matrix = do_tfidf(data_test)

# LSA

In [208]:
from scipy.sparse.linalg import svds

def do_lsa(data):
    num_components = 10
    q, s, p = svds(tfidf, k = num_components)

    lsa_doc_matrix = pd.DataFrame(data=q)

    #display(lsa_doc_matrix)

    return(lsa_doc_matrix)

# call function
train_lsa_matrix = do_lsa(train_tfidf_matrix)
test_lsa_matrix = do_lsa(test_tfidf_matrix)

# Doc2Vec

In [209]:
import gensim
from gensim.models.doc2vec import Doc2Vec

def do_doc2vec(data):
    tagged_documents = []
    sentences = [text.split() for text in data['CleanText']]

    for i, doc in enumerate(sentences):
        tagged_documents.append(gensim.models.doc2vec.TaggedDocument(doc, [i]))

    d2v = Doc2Vec(vector_size=50, min_count=2, epochs=40)
    d2v.build_vocab(tagged_documents)

    d2v.train(tagged_documents, total_examples=d2v.corpus_count, epochs=d2v.epochs)

    doc2vec_alldocs = []
    for i in range(len(tagged_documents)):
        doc2vec_alldocs.append(d2v.infer_vector(tagged_documents[i].words))

    doc2vec_alldocs_matrix = pd.DataFrame(doc2vec_alldocs)

    #display(doc2vec_alldocs_matrix)
    
    return(doc2vec_alldocs_matrix)

# call function
train_doc2vec_matrix = do_doc2vec(data_train) 
test_doc2vec_matrix = do_doc2vec(data_test)

# Generating a Neural Network